As you will see, the microarray 'processed' data obtained from the array express dataset is not really processed.

**data ref:** https://jb.asm.org/content/190/11/3904

In [1]:
import pandas as pd
import re

In [2]:
data_file_path = 'ArrayExpressData/E-TABM-386/'
processed_matrix = 'FinalDataMatrix.txt'

toepel_raw = pd.read_csv(data_file_path+processed_matrix,sep='\t',header=[0,1])
toepel_raw.head()

Scan REF  \
                Reporter REF   
0  Contig0.10_10_14370_15446   
1  Contig0.10_11_16336_15422   
2  Contig0.10_12_16503_17552   
3  Contig0.10_13_17679_17981   
4  Contig0.10_14_18000_18698   

       PooledBioReps_Day2_LL6_1;PooledBioReps_Day2_LL6_2  \
  Software Unknown:Replicates Mean Normalized Log2 Ratio   
0                                           0.755742       
1                                           0.289256       
2                                          -0.270551       
3                                          -0.082366       
4                                          -0.062873       

                                                                 \
  Software Unknown:Replicates SD Software Unknown:Replicates SE   
0                       0.160192                       0.080096   
1                       0.156415                       0.078208   
2                       0.056849                       0.028424   
3                       0.082032                       0.041016   
4                       0.103646                       0.051823   

       PooledBioReps_Day2_LL2_1;PooledBioReps_Day2_LL2_2  \
  Software Unknown:Replicates Mean Normalized Log2 Ratio   
0                                           0.389711       
1                                           0.030779       
2                                           0.368210       
3                                           0.194944       
4                                           0.162255       

                                                                 \
  Software Unknown:Replicates SD Software Unknown:Replicates SE   
0                       0.100193                       0.050097   
1                       0.135529                       0.067765   
2                       0.057689                       0.028845   
3                       0.058718                       0.029359   
4                       0.034064                       0.017032   

         PooledBioReps_Day1_L2_1;PooledBioReps_Day1_L2_2  \
  Software Unknown:Replicates Mean Normalized Log2 Ratio   
0                                           0.119984       
1                                          -0.006948       
2                                           0.334171       
3                                           0.418948       
4                                           0.324850       

                                                                 ...  \
  Software Unknown:Replicates SD Software Unknown:Replicates SE  ...   
0                       0.099797                       0.049898  ...   
1                       0.035033                       0.017517  ...   
2                       0.016476                       0.008238  ...   
3                       0.068138                       0.034069  ...   
4                       0.060975                       0.030488  ...   

  PooledBioReps_Day2_LL10_1;PooledBioReps_Day2_LL10_2  \
                       Software Unknown:Replicates SE   
0                                           0.054582    
1                                           0.070803    
2                                           0.065553    
3                                           0.012190    
4                                           0.043217    

       PooledBioReps_Day2_LD2_1;PooledBioReps_Day2_LD2_2  \
  Software Unknown:Replicates Mean Normalized Log2 Ratio   
0                                          -0.620710       
1                                          -0.285271       
2                                           0.252908       
3                                           0.052271       
4                                           0.090790       

                                                                 \
  Software Unknown:Replicates SD Software Unknown:Replicates SE   
0                       0.074663                       0.037332   
1                       0.142685                       0.071342  

The columns are all messed up (starts from Day2 LL9), index has contigs (we'll get proper gene names), the rest of the data looks good (it is already normalized and log2 values are calculated, sd and se are also given). 

We will arrange the columns referring the paper (link given above). The order will be Day1_L2, Day1_L6, Day1_L10, Day1_D2, Day1_D6, Day1_D10, Day2_LL2, Day2_LL6, Day2_LL10, Day2_LD2, Day2_LD6, Day2_LD10.

The column has 2 levels, we'll edit the column names in both the levels and follow the above pattern.  

In [3]:
def colNameFunc(name):
    names = name.split(';')
    pattern = re.compile(r'^PooledBioReps_(Day[1|2])_([D|L]{1,2}[0-9]{1,2})_[1|2]$')
    matched = re.match(pattern,names[0])
    
    return '_'.join([matched.group(1),matched.group(2)]) if matched else names[0]

def colNameLevel1(name):
    names = name.split(':')
    if len(names)==1:
        return names[0]
    replace = {'Replicates Mean Normalized Log2 Ratio': 'Exp-Val','Replicates SD': 'SD', 'Replicates SE':'SE'}
    return replace[names[1]]

toepel_new_col_names = list(map(colNameFunc,list(toepel_raw.columns.levels[0])))
toepel_new_col_level1 = list(map(colNameLevel1,list(toepel_raw.columns.levels[1])))

In [4]:
toepel_raw.columns = toepel_raw.columns.set_levels(toepel_new_col_names,level=0)
toepel_raw.columns = toepel_raw.columns.set_levels(toepel_new_col_level1,level=1)

Next we will only keep the log2 ratio column and the contigs, and merge into a single column level.

In [5]:
df = toepel_raw.loc[:,toepel_raw.columns.get_level_values(1).isin({'Exp-Val','Reporter REF'})]

Then we will sort the column order as given above.

In [6]:
df.columns = df.columns.droplevel(1)

In [7]:
df = df.reindex(['Day1_L2', 'Day1_L6', 'Day1_L10', 'Day1_D2', 'Day1_D6', 'Day1_D10', 'Day2_LL2', 'Day2_LL6', 
                 'Day2_LL10', 'Day2_LD2', 'Day2_LD6', 'Day2_LD10','Scan REF'],axis=1).set_index('Scan REF')

In [8]:
df.head()

Day1_L2   Day1_L6  Day1_L10   Day1_D2   Day1_D6  \
Scan REF                                                                      
Contig0.10_10_14370_15446  0.119984  0.574008 -0.497268 -0.946784 -0.912688   
Contig0.10_11_16336_15422 -0.006948  0.377326  0.281436  0.096787  0.101432   
Contig0.10_12_16503_17552  0.334171 -0.366012  0.161679 -0.743794 -0.841029   
Contig0.10_13_17679_17981  0.418948 -0.146916 -0.605930 -0.099442 -0.115359   
Contig0.10_14_18000_18698  0.324850  0.036866 -0.001216 -0.144987  0.029886   

                           Day1_D10  Day2_LL2  Day2_LL6  Day2_LL10  Day2_LD2  \
Scan REF                                                                       
Contig0.10_10_14370_15446 -0.555053  0.389711  0.755742  -0.552432 -0.620710   
Contig0.10_11_16336_15422 -0.168668  0.030779  0.289256   0.232399 -0.285271   
Contig0.10_12_16503_17552 -0.762609  0.368210 -0.270551   0.121346  0.252908   
Contig0.10_13_17679_17981  0.130870  0.194944 -0.082366  -0.290507  0.052271   
Contig0.10_14_18000_18698  0.111226  0.162255 -0.062873   0.064338  0.090790   

                           Day2_LD6  Day2_LD10  
Scan REF                                        
Contig0.10_10_14370_15446  0.161136   0.618904  
Contig0.10_11_16336_15422  0.064953   0.001840  
Contig0.10_12_16503_17552  0.181769   0.212853  
Contig0.10_13_17679_17981  0.094243   0.111787  
Contig0.10_14_18000_18698  0.173198   0.289154

Next we will map the contigs to the gene names. For that, we will need the contig to gene mappings.

In [9]:
mappings = 'Contig-ORF.txt'
toepel_contig = pd.read_csv(data_file_path+mappings,sep='\t',header=None)
toepel_contig.columns = ['Contig','ORF']

In [10]:
toepel_contig.head()

Contig       ORF
0  ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-951.Cont...  cce_4187
1  ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-951.Cont...  cce_4186
2  ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-951.Cont...  cce_4185
3  ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-951.Cont...  cce_4184
4  ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-951.Cont...  cce_4183

Need to parse the contig columns properly and apply the function to all the rows of that columns. 

In [11]:
def ContigFunc(name):
    return name.replace('ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-951.','')

toepel_contig.Contig = toepel_contig.Contig.apply(ContigFunc)
toepel_contig = toepel_contig.iloc[:-1].set_index('Contig')

Creating a new dataframe by merging information from both the dataframes

In [12]:
new_df = toepel_contig.join(df)

In [13]:
new_df.head()

ORF   Day1_L2   Day1_L6  Day1_L10   Day1_D2  \
Contig                                                                        
Contig0.10_10_14370_15446  cce_4187  0.119984  0.574008 -0.497268 -0.946784   
Contig0.10_11_16336_15422  cce_4186 -0.006948  0.377326  0.281436  0.096787   
Contig0.10_12_16503_17552  cce_4185  0.334171 -0.366012  0.161679 -0.743794   
Contig0.10_13_17679_17981  cce_4184  0.418948 -0.146916 -0.605930 -0.099442   
Contig0.10_14_18000_18698  cce_4183  0.324850  0.036866 -0.001216 -0.144987   

                            Day1_D6  Day1_D10  Day2_LL2  Day2_LL6  Day2_LL10  \
Contig                                                                         
Contig0.10_10_14370_15446 -0.912688 -0.555053  0.389711  0.755742  -0.552432   
Contig0.10_11_16336_15422  0.101432 -0.168668  0.030779  0.289256   0.232399   
Contig0.10_12_16503_17552 -0.841029 -0.762609  0.368210 -0.270551   0.121346   
Contig0.10_13_17679_17981 -0.115359  0.130870  0.194944 -0.082366  -0.290507   
Contig0.10_14_18000_18698  0.029886  0.111226  0.162255 -0.062873   0.064338   

                           Day2_LD2  Day2_LD6  Day2_LD10  
Contig                                                    
Contig0.10_10_14370_15446 -0.620710  0.161136   0.618904  
Contig0.10_11_16336_15422 -0.285271  0.064953   0.001840  
Contig0.10_12_16503_17552  0.252908  0.181769   0.212853  
Contig0.10_13_17679_17981  0.052271  0.094243   0.111787  
Contig0.10_14_18000_18698  0.090790  0.173198   0.289154

In [14]:
processed_data_toepel = new_df.reset_index()

In [15]:
processed_data_toepel.head()

Contig       ORF   Day1_L2   Day1_L6  Day1_L10  \
0  Contig0.10_10_14370_15446  cce_4187  0.119984  0.574008 -0.497268   
1  Contig0.10_11_16336_15422  cce_4186 -0.006948  0.377326  0.281436   
2  Contig0.10_12_16503_17552  cce_4185  0.334171 -0.366012  0.161679   
3  Contig0.10_13_17679_17981  cce_4184  0.418948 -0.146916 -0.605930   
4  Contig0.10_14_18000_18698  cce_4183  0.324850  0.036866 -0.001216   

    Day1_D2   Day1_D6  Day1_D10  Day2_LL2  Day2_LL6  Day2_LL10  Day2_LD2  \
0 -0.946784 -0.912688 -0.555053  0.389711  0.755742  -0.552432 -0.620710   
1  0.096787  0.101432 -0.168668  0.030779  0.289256   0.232399 -0.285271   
2 -0.743794 -0.841029 -0.762609  0.368210 -0.270551   0.121346  0.252908   
3 -0.099442 -0.115359  0.130870  0.194944 -0.082366  -0.290507  0.052271   
4 -0.144987  0.029886  0.111226  0.162255 -0.062873   0.064338  0.090790   

   Day2_LD6  Day2_LD10  
0  0.161136   0.618904  
1  0.064953   0.001840  
2  0.181769   0.212853  
3  0.094243   0.111787  
4  0.173198   0.289154

*Will use this data for analysis*

In [16]:
processed_data_toepel.to_csv('MicroarrayData/ToepelProcessed.csv',index=False)